In [315]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

import pandas as pd

In [316]:
chrome = webdriver.Chrome()
chrome.get("https://www.panganku.org/en-EN/semua_nutrisi")
chrome.implicitly_wait(10)

In [317]:
foods = []

In [321]:
def get_food_data(driver, table_row):
    data = {}
    field_names = ["Food Code", "Food Name", "Group", "Type"]

    for i in range(4):
        element = driver.find_element(By.XPATH, f"//*[@id=\"data\"]/tbody/tr[{table_row}]/td[{i + 2}]")
        data[field_names[i]] = element.text

    return data


def get_nutrition_compositions_data(driver):
    data = {}

    table_of_nutritions = driver.find_elements(By.XPATH, '//*[@id="print-area"]/div[2]/section/div/div')

    for i in range(len(table_of_nutritions)):
        nutritions = table_of_nutritions[i].find_elements(By.XPATH, './div/div/table/tbody/tr')

        for j in range(len(nutritions)):
            result = list(map(lambda x: x.strip(), nutritions[j].text.split(":")))
            if len(result) == 2:
                value = result[1].split(" ")
                
                data[f"{result[0]} ({value[1]})"] = float(value[0].replace(",", "."))

    return data


In [323]:
data_length_select = Select(chrome.find_element(By.XPATH, '//*[@id="data_length"]/label/select'))
data_length_select.select_by_value("-1")

In [324]:
table_of_foods = chrome.find_elements(By.XPATH, '//*[@id="data"]/tbody/tr')

for i in range(len(table_of_foods)):
    food = get_food_data(chrome, i + 1)

    data = chrome.find_element(By.XPATH, f"//*[@id='data']/tbody/tr[{i + 1}]")
    data.click()

    nutrition_compositions = get_nutrition_compositions_data(chrome)
    food.update(nutrition_compositions)
    # food_df.concat(food, ignore_index=True)
    foods.append(food)

    print(f"{i + 1}: {food['Food Code']} | {food['Food Name']}")

    chrome.back()

1: HR005 | Alabio duck egg, fresh
2: DR037 | Ambarella leaves, fresh
3: ER074 | Ambon banana, fresh
4: DR072 | Ambon cassave leaves, fresh
5: DR054 | Ambon matel leaves, fresh
6: DR050 | American black nightshade leaves, fresh
7: AP026 | Amparan tatak
8: DR073 | Ampenan cassava leaves, fresh
9: GP041 | Anchovy, dried, raw
10: GP042 | Anchovy, dried, unsalted, raw
11: GR078 | Anchovy, fresh
12: GP043 | Anchovy, ground, raw


KeyboardInterrupt: 

In [325]:
food_df = pd.DataFrame.from_dict(foods)

In [314]:
chrome.quit()

In [328]:
food_df

,Food Code,Food Name,Group,Type,Air (Water) g,Energi (Energy) Kal,Protein (Protein) g,Lemak (Fat) g,Karbohidrat (CHO) g,Serat (Fibre) g,...,Kalium (K) mg,Tembaga (Cu) mg,Seng (Zn) mg,Retinol (Vit. A) mcg,Beta-Karoten (Carotenes) mcg,Thiamin (Vit. B1) mg,Riboflavin (Vit. B2) mg,Niasin (Niacin) mg,Karoten Total (Re) mcg,Vitamin C (Vit. C) mg
0,HR005,"Alabio duck egg, fresh",Eggs,Raw,70.0,187.0,11.8,14.2,3.0,0.0,...,80.0,0.6,1.7,180.0,437.000,1.55,0.37,0.1,NaN,NaN
1,HR005,"Alabio duck egg, fresh",Eggs,Raw,70.0,187.0,11.8,14.2,3.0,0.0,...,80.0,0.6,1.7,180.0,437.000,1.55,0.37,0.1,NaN,NaN
2,DR037,"Ambarella leaves, fresh",Vegetables,Raw,81.0,59.0,3.5,0.3,13.4,10.7,...,579.0,NaN,NaN,NaN,1.624,0.06,0.20,1.5,2.9,29.0
3,ER074,"Ambon banana, fresh",Fruits,Raw,72.9,108.0,1.0,0.8,24.3,1.9,...,0.0,0.2,0.2,NaN,37.000,0.05,0.11,0.1,NaN,9.0
4,DR072,"Ambon cassave leaves, fresh",Vegetables,Raw,58.0,144.0,12.7,1.2,25.2,10.2,...,62.2,0.2,5.4,NaN,8.627,0.21,1.00,4.2,13.0,220.0
5,DR054,"Ambon matel leaves, fresh",Vegetables,Raw,77.1,71.0,6.8,1.6,11.5,4.6,...,NaN,NaN,NaN,0.0,NaN,0.08,NaN,NaN,49.0,26.0
6,DR050,"American black nightshade leaves, fresh",Vegetables,Raw,85.0,49.0,4.7,0.5,8.1,NaN,...,NaN,NaN,NaN,NaN,NaN,0.14,NaN,NaN,1.9,40.0
7,AP026,Amparan tatak,Cereals,Processed/Products,58.4,191.0,1.3,5.5,34.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,DR073,"Ampenan cassava leaves, fresh",Vegetables,Raw,84.4,50.0,6.2,1.1,7.1,2.4,...,570.0,0.2,1.3,NaN,7.917,0.50,0.10,1.6,NaN,103.0
9,GP041,"Anchovy, dried, raw",Fish and Other Seafood,Processed/Products,15.0,277.0,60.0,2.3,1.8,0.0,...,NaN,NaN,NaN,90.0,NaN,0.10,NaN,NaN,NaN,0.0


In [ ]:
food_df.to_csv("food_data.csv", index=False)